In [1]:
using JuMP
using CPLEX
using Distributions
using MathProgBase

In [2]:
srand(1) #para usar una única semilla y repetir siempre los mismos números aleatorios

A=[44.7  1411   2.0   365  0     55.4  33.3  441  0;
    36   897    1.7   99   30.9  17.4  7.9   106  0;
    8.4  422    15.1  9    26    3     23.5  11   60;
    20.6 17     0.6   6    55.8  0.2   0     0    0;
    7.4  448    16.4  19   28.1  0.8   10.3  4    0;
    15.7 661    1     48   0     9.6   8.1   471  0;
    41.7 0      0     0    0.2   0     0.5   5    0;
    2.2  333    0.2   139  169.2 6.4   50.8  316  525;
    4.4  249    0.3   37   0     18.2  3.6   79   0;
    5.8  705    6.8   45   3.5   1     4.9   209  0;
    2.4  138    3.7   80   69    4.3   5.8   37   862;
    2.6  125    4     36   7.2   9     4.5   26   5369;
    5.8  166    3.8   59   16.6  4.7   5.9   21   1184;
    14.3 336    1.8   118  6.7   29.4  7.1   198  2522;
    1.1  106    0     138  918.4 5.7   13.8  33   2755;
    9.6  138    2.7   54   290.7 8.4   5.4   83   1912;
    8.5  87     1.7   173  86.8  1.2   4.3   55   57;
    12.8 99     2.5   154  85.7  3.9   4.3   65   257;
    17.4 1055   3.7   459  5.1   26.9  38.2  93   0;
    26.9 1691   11.4  792  0     38.4  24.6  217  0]

A=A';

b=[30,70,0.8,12,5,1.8,2.7,18,75]

c=ones(20,1);

function randomcoef(c)
    cl=zeros(length(c));
    cu=zeros(length(c));
    for i=1:length(c)
        cl[i]=c[i]-(c[i]*rand()/2);
        cu[i]=c[i]+(c[i]*rand()/2);
    end
    return cl,cu
end
c_min,c_max=randomcoef(c)
println("c_min: ",c_min)
println("c_max: ",c_max)

c_min: [0.881983,0.843647,0.755694,0.524042,0.874169,0.722124,0.787641,0.859405,0.87431,0.856149,0.961525,0.563228,0.624344,0.961087,0.957182,0.768325,0.94401,0.974193,0.772154,0.910877]
c_max: [1.17326,1.00395,1.10548,1.49995,1.49333,1.21855,1.38661,1.10474,1.01019,1.42976,1.3202,1.13929,1.32244,1.42409,1.2766,1.09291,1.48816,1.26901,1.1397,1.27449]


In [3]:
function buildinstance(n,m)
    #Vector de costos de primera etapa
    c=zeros(n)
    for i=1:n
        min=round(rand()*10)
        max=min+1+round(rand()*min*10)
        c[i]=round(rand(Uniform(min,max)))
        if c[i]==0
            c[i]=1
        end
    end

    #A matrix
    A=round(rand(1,n)*rand(Uniform(1,30)))
    for i=2:m
        min=round(rand()*rand(Uniform(1,30)))
        max=min+1+round(rand()*min*rand(Uniform(1,30)))
        A=vcat(A,round(rand(Uniform(min,max),1,n)))
    end

    #Set b vector
    incap=sum(A,2)
    b=round((incap.*round(rand(Uniform(5,20),m,1))).*rand(m))
    return c,A,b
end

buildinstance (generic function with 1 method)

In [ ]:
c,A,b=buildinstance(10000,1000)
c_min,c_max=randomcoef(c)
b=b/10000

In [20]:
function kasperskinstance(n,m)
    #Vector de costos de primera etapa
    cl=zeros(n)
    cu=zeros(n)
    for i=1:n
        M=1+round(rand()*100)
        min=round(rand(Uniform(1,M+1)))
        max=min+round(rand(Uniform(1,M+1)))
        cl[i]=min
        cu[i]=max
    end

    #A matrix
    A=round(rand(1,n)*rand(Uniform(1,30)))
    for i=2:m
        min=round(rand()*rand(Uniform(1,30)))
        max=min+1+round(rand()*min*rand(Uniform(1,30)))
        A=vcat(A,round(rand(Uniform(min,max),1,n)))
    end

    #Set b vector
    incap=sum(A,2)
    b=round((incap.*round(rand(Uniform(5,20),m,1))).*rand(m))
    return cl,cu,A,b
end
c_min,c_max,A,b=kasperskinstance(1000,400)
b=b/1000
println(c_min)
println(c_max)

#preproceso para determinar límites superiores
xmax=zeros(length(c_max));
for j=1:length(c_max)
    v=zeros(length(c_max));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    #valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    #Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    #pos=find(x->x==xm,b./A[:,j])
    #pos1=find(x->x==xm,maxim)
    #new=xm-Ar[pos]
    #if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
    #    xmax[j]=new[1]
    #else
    #    xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    #end
    xmax[j]=xm
end
println(xmax)

[32

.0,61.0,20.0,81.0,45.0,1.0,33.0,38.0,39.0,13.0,53.0,27.0,13.0,62.0,15.0,17.0,85.0,4.0,76.0,27.0,59.0,37.0,62.0,21.0,8.0,7.0,3.0,26.0,29.0,7.0,24.0,34.0,15.0,29.0,6.0,10.0,47.0,63.0,33.0,59.0,69.0,5.0,4.0,2.0,3.0,76.0,40.0,22.0,25.0,16.0,50.0,2.0,1.0,26.0,7.0,56.0,6.0,9.0,1.0,25.0,3.0,29.0,17.0,9.0,73.0,28.0,10.0,61.0,26.0,2.0,70.0,34.0,10.0,78.0,2.0,18.0,1.0,41.0,38.0,61.0,30.0,38.0,24.0,2.0,26.0,19.0,75.0,33.0,60.0,6.0,11.0,28.0,41.0,10.0,7.0,17.0,34.0,41.0,2.0,9.0,5.0,6.0,84.0,17.0,68.0,10.0,8.0,42.0,16.0,25.0,16.0,9.0,42.0,51.0,36.0,5.0,17.0,53.0,35.0,4.0,36.0,14.0,28.0,21.0,6.0,3.0,5.0,11.0,74.0,4.0,9.0,26.0,16.0,20.0,20.0,62.0,38.0,71.0,2.0,3.0,3.0,65.0,1.0,21.0,54.0,38.0,71.0,8.0,50.0,9.0,3.0,10.0,13.0,9.0,42.0,4.0,53.0,5.0,43.0,41.0,14.0,2.0,19.0,12.0,48.0,76.0,2.0,81.0,30.0,43.0,46.0,48.0,5.0,63.0,36.0,8.0,50.0,18.0,18.0,63.0,76.0,43.0,5.0,11.0,24.0,18.0,3.0,33.0,9.0,22.0,10.0,54.0,23.0,7.0,52.0,76.0,17.0,51.0,6.0,57.0,49.0,16.0,30.0,12.0,2.0,5.0,14.0,16.0,52.0,6.0,46.0,23.0,8.

[47.0,137.0,56.0,109.0,66.0,7.0,62.0,45.0,73.0,30.0,102.0,70.0,19.0,122.0,22.0,46.0,116.0,9.0,119.0,58.0,91.0,41.0,113.0,57.0,12.0,13.0,4.0,71.0,67.0,26.0,42.0,60.0,24.0,54.0,8.0,27.0,67.0,119.0,91.0,79.0,112.0,11.0,8.0,10.0,32.0,125.0,93.0,32.0,68.0,35.0,87.0,51.0,20.0,32.0,13.0,116.0,28.0,23.0,5.0,56.0,32.0,44.0,49.0,18.0,134.0,45.0,19.0,97.0,49.0,17.0,142.0,119.0,13.0,123.0,18.0,34.0,33.0,82.0,94.0,92.0,42.0,106.0,45.0,5.0,90.0,50.0,172.0,43.0,107.0,9.0,24.0,58.0,85.0,19.0,17.0,55.0,75.0,54.0,15.0,24.0,7.0,11.0,97.0,28.0,166.0,27.0,29.0,75.0,35.0,66.0,23.0,16.0,84.0,104.0,79.0,7.0,30.0,86.0,53.0,5.0,79.0,42.0,54.0,24.0,21.0,5.0,29.0,48.0,132.0,9.0,11.0,41.0,20.0,26.0,43.0,88.0,80.0,81.0,4.0,6.0,5.0,128.0,13.0,27.0,86.0,69.0,111.0,16.0,70.0,74.0,5.0,12.0,74.0,18.0,82.0,17.0,115.0,12.0,45.0,58.0,25.0,16.0,24.0,13.0,71.0,78.0,39.0,148.0,55.0,94.0,71.0,113.0,27.0,104.0,69.0,43.0,82.0,88.0,24.0,83.0,96.0,89.0,11.0,27.0,68.0,57.0,6.0,67.0,58.0,43.0,19.0,57.0,82.0,23.0,127.0,87.0,19.0,58.0

[85.025,66.9838,137.159,85.4129,62.4641,121.88,99.7201,86.7229,78.6107,88.1736,71.1774,69.235,66.9259,118.332,64.97,130.225,83.5955,67.4312,85.1784,67.3673,62.3532,121.88,68.2215,55.1873,76.5288,49.86,78.3354,58.457,85.025,98.6928,121.88,137.115,101.051,75.7102,65.3489,133.968,92.8208,101.177,91.574,58.713,101.051,91.5138,90.7133,74.7485,94.4478,121.412,129.539,131.497,153.528,61.3187,106.288,74.7067,70.3962,138.802,101.573,62.6615,94.9562,85.379,70.8641,60.706,94.6654,58.236,88.1736,70.1484,94.6654,94.2936,149.413,102.887,49.3138,100.476,85.025,70.031,92.8208,120.207,67.162,132.812,49.3464,92.8208,101.051,72.6135,85.025,58.713,118.332,101.051,74.7485,100.556,101.573,70.031,80.3821,121.412,101.573,91.5138,148.614,58.713,69.235,66.5392,121.593,137.159,57.0523,64.7695,118.332,118.332,82.4349,108.628,112.221,76.56,63.774,131.497,87.7015,82.2315,99.2052,157.776,132.812,51.9372,52.2893,50.7865,76.56,154.304,67.3673,60.005,77.9427,154.304,78.2122,48.7102,79.1257,101.051,67.6181,52.5919,70.39

In [29]:
function Montemanninstance(n,m)
    #Vector de costos de primera etapa
    cl=zeros(n)
    cu=zeros(n)
    for i=1:n
        M=1+round(rand()*100)
        min=round(rand(Uniform(1,M+1)))
        cl[i]=min
        cu[i]=M+1
    end

    #A matrix
    A=round(rand(1,n)*rand(Uniform(1,200)))
    for i=2:m
        min=round(rand()*rand(Uniform(1,200)))
        max=min+1+round(rand()*min*rand(Uniform(1,200)))
        A=vcat(A,round(rand(Uniform(min,max),1,n)))
    end

    #Set b vector
    incap=sum(A,2)
    b=round((incap.*round(rand(Uniform(5,20),m,1))).*rand(m))
    return cl,cu,A,b
end
c_min,c_max,A,b=Montemanninstance(1000,400)
b=b/1000
println(c_min)
println(c_max)

#preproceso para determinar límites superiores
xmax=zeros(length(c_max));
for j=1:length(c_max)
    v=zeros(length(c_max));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    #valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    #Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    #pos=find(x->x==xm,b./A[:,j])
    #pos1=find(x->x==xm,maxim)
    #new=xm-Ar[pos]
    #if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
    #    xmax[j]=new[1]
    #else
    #    xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    #end
    xmax[j]=xm
end
println(xmax)

[28.0,29.0,5.0,34.0,15.0,9.0,6.0,13.0,1.0,56.0,10.0,5.0,79.0,5.0,7.0,35.0,22.0,14.0,71.0,8.0,2.0,67.0,26.0,40.0,25.0,27.0,55.0,47.0,36.0,6.0,24.0,13.0,42.0,44.0,14.0,49.0,9.0,21.0,10.0,12.0,45.0,38.0,33.0,4.0,43.0,52.0,9.0,16.0,68.0,6.0,7.0,37.0,57.0,3.0,10.0,7.0,11.0,22.0,1.0,47.0,12.0,69.0,36.0,3.0,29.0,5.0,2.0,9.0,5.0,47.0,2.0,80.0,41.0,51.0,47.0,10.0,2.0,3.0,48.0,34.0,30.0,41.0,43.0,53.0,25.0,47.0,14.0,3.0,46.0,2.0,32.0,2.0,30.0,3.0,71.0,24.0,29.0,50.0,88.0,12.0,8.0,37.0,11.0,8.0,19.0,6.0,3.0,19.0,15.0,51.0,5.0,72.0,2.0,29.0,27.0,26.0,70.0,3.0,19.0,27.0,20.0,29.0,52.0,68.0,56.0,33.0,11.0,19.0,24.0,14.0,22.0,64.0,65.0,31.0,62.0,47.0,11.0,17.0,15.0,14.0,53.0,1.0,3.0,13.0,18.0,12.0,20.0,40.0,9.0,13.0,39.0,42.0,8.0,50.0,15.0,21.0,78.0,50.0,5.0,23.0,35.0,3.0,15.0,5.0,5.0,9.0,4.0,5.0,30.0,12.0,5.0,15.0,11.0,26.0,86.0,26.0,17.0,57.0,49.0,68.0,56.0,37.0,15.0,2.0,36.0,47.0,24.0,43.0,7.0,36.0,37.0,39.0,23.0,4.0,19.0,41.0,14.0,70.0,15.0,29.0,18.0,21.0,3.0,36.0,5.0,23.0,9.0,23.0,13.0,12.0,43.0

.0,64.0,7.0,20.0,1.0,50.0,31.0,3.0,93.0,7.0,25.0,42.0,3.0,39.0,14.0,62.0,6.0,46.0,27.0,40.0,33.0,1.0,9.0,42.0,58.0,14.0,33.0,10.0,29.0,27.0,23.0,37.0,2.0,64.0,63.0,7.0,23.0,30.0,19.0,38.0,30.0,72.0,7.0,49.0,17.0,21.0,5.0,1.0,33.0,37.0,82.0,35.0,10.0,3.0,13.0,18.0,10.0,18.0,5.0,38.0,1.0,8.0,55.0,7.0,21.0,9.0,79.0,77.0,4.0,46.0,17.0,31.0,18.0,55.0,49.0,32.0,7.0,5.0,3.0,23.0,40.0,8.0,44.0,48.0,69.0,45.0,40.0,30.0,44.0,39.0,1.0,21.0,33.0,43.0,16.0,37.0,13.0,26.0,14.0,83.0,8.0,73.0,42.0,27.0,50.0,2.0,15.0,50.0,17.0,17.0,7.0,48.0,55.0,6.0,84.0,3.0,10.0,15.0,5.0,62.0,31.0,95.0,39.0,26.0,53.0,14.0,55.0,6.0,31.0,3.0,1.0,4.0,16.0,11.0,26.0,12.0,8.0,8.0,45.0,13.0,3.0,29.0,19.0,9.0,8.0,6.0,3.0,15.0,4.0,6.0,74.0,4.0,54.0,39.0,83.0,32.0,2.0,21.0,4.0,22.0,2.0,10.0,2.0,9.0,48.0,85.0,8.0,11.0,10.0,9.0,36.0,5.0,7.0,31.0,10.0,43.0,4.0,78.0,8.0,6.0,13.0,52.0,6.0,28.0,76.0,7.0,4.0,28.0,89.0,13.0,56.0,56.0,5.0,2.0,44.0,7.0,25.0,47.0,12.0,11.0,36.0,35.0,36.0,25.0,81.0,19.0,5.0,45.0,51.0,5.0,7.0,9.0,61.0,19.0

[204.127,383.573,477.301,219.454,115.682,136.682,305.139,329.496,124.989,177.806,166.576,405.49,456.683,225.609,426.87,155.763,207.516,335.958,187.802,137.071,105.046,177.99,157.938,133.77,139.051,379.874,167.679,254.282,244.063,256.468,259.704,348.56,176.042,132.153,200.428,160.31,489.539,140.615,181.191,368.778,172.92,269.383,189.744,309.452,122.385,575.569,247.422,179.205,534.757,105.0,503.938,159.348,216.197,633.357,363.354,360.878,248.438,262.359,631.693,204.667,199.692,386.385,403.852,330.588,287.68,159.663,227.717,262.164,159.017,143.258,460.624,188.244,489.539,300.65,204.563,526.675,106.795,148.99,332.792,224.218,460.624,101.89,310.729,254.694,358.356,240.899,328.192,237.639,375.551,262.461,230.637,156.144,546.188,166.972,445.399,281.492,647.547,444.133,310.969,134.378,94.2237,165.596,142.473,206.515,185.152,220.392,233.336,314.058,613.69,210.516,460.268,115.522,99.3502,142.353,323.439,115.682,217.815,208.755,190.407,168.324,198.866,384.344,212.436,471.711,85.4894,493.002,265.2

In [9]:
function solveScenario(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end


solveScenario (generic function with 1 method)

In [16]:
function solveLP(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

#Subrutina para fijar el límite superior
function solveLP(A,b,c,solver,v,max)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    @constraint(submodel, sum(v[j]*x[j] for j in 1:n)==max)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor,sol=getobjectivevalue(submodel),getvalue(x)
    return valor,sol
end
#preproceso para determinar límites superiores
xmax=zeros(length(c));
for j=1:length(c)
    v=zeros(length(c));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    #valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    #Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    #pos=find(x->x==xm,b./A[:,j])
    #pos1=find(x->x==xm,maxim)
    #new=xm-Ar[pos]
    #if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
    #    xmax[j]=new[1]
    #else
    #    xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    #end
    xmax[j]=xm
end
println(xmax)

[501.026,1002.05,424.219,512.137,487.138,466.579,287.613,480.678,433.347,318.164,480.678,486.359,268.313,520.366,641.304,313.407,298.856,267.283,501.026,303.001]


In [12]:
#Subproblem (NP-hard) function 
function SubproblemLP(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)

    #function infocallback(cb)
    #    node      = MathProgBase.cbgetexplorednodes(cb)
    #    obj       = MathProgBase.cbgetobj(cb)
    #    bestbound = MathProgBase.cbgetbestbound(cb)
    #    push!(bbdata, NodeData(time(),node,obj,bestbound))
    #end
    #addinfocallback(m, infocallback, when = :Intermediate)
    function infocallback(cb)
        #obj       = MathProgBase.cbgetobj(cb)
        println("\n************************ PIPA\n ")
    end
    addlazycallback(m, infocallback)

    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            #println("upper")
            setlowerbound(zup[j],1) #fuerza a que valga 1
            setupperbound(zun[j],0) #fuerza a que valga 0
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setupperbound(zup[j],0)
                setlowerbound(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    status = solve(m)
    print("solucion: ",status,"\t",getobjectivevalue(m),"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end    
    return valor,cost_scen
end

SubproblemLP (generic function with 1 method)

In [ ]:
function solveRLP(A,b,cminus, cplus, xmax, n::Int64, solver=CplexSolver(CPX_PARAM_LPMETHOD=1,CPX_PARAM_SCRIND=0))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveScenario(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    #rhs = - optimalCost  [JPG]
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    #@constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)   [JPG]
    @constraint(m,θ >= sum(cMidPoint[j]*x[j] for j in 1:n) - optimalCost  )
    #print(m)
    

    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB ",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol #conteo<=10 # [JPG]
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\tfixup: ",fixup,"\n")
        #println("\t\tfixun: ",fixun,"\n")
        tt=time()
#        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b)
        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b)
        #println("Tiempo subproblema: ", time()-tt)
        print("Costo optimo subproblema: ", optimalCost)
        println(" vector cost: ",cost)
        #New cut
        tt1=time()
        rhs=solveScenario(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("Tiempo RHS: ", time()-tt1)
        #@constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs ) [JPG]
        @constraint(m,θ >= sum(cost[j]*x[j] for j in 1:n) - rhs  )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        #println("Tiempo LP: ", time()-tt2)
        println(status)
        println("θ ",getvalue(θ),"\tvariables master: ",getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        #println("LB: ", getobjectivevalue(m))
        it += 1
        println("GAP entre soluciones consecutivas ", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

In [ ]:
solveRLP(A,b,c_min, c_max, xmax, length(c))

In [26]:
type NodeData
    time::Float64  # in seconds since the epoch
    obj::Float64
end

#Subproblem (NP-hard) function 
function SubproblemLPim(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b,zopt,tt,scens,timemax,mindis)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    function infocallback(cb)
        #obj       = MathProgBase.cbgetobj(cb)
        println("\n************************ PIPA\n ")
    end
    addlazycallback(m, infocallback)
    
    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            setvalue(zup[j],1)
            setvalue(zun[j],0)
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setvalue(zup[j],0)
                setvalue(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    #function getcursol(cb)
    #    zval=getvalue(zup)
    #    obj=MathProgBase.cbgetobj(cb)
    #    cost_scen=zeros(length(zval))
    #    println("Solución parcial: ", obj)
    #    println("zval ", zval)
    #    
    #    intsol, cost_scen = check_solution(zval,cun,cup)
    #    println("Es entera la solución:", intsol)
    #    
    #    setdis = check_distance(scens,cost_scen,mindis)
    #    println("Cumple condiciones de distancia: ", setdis)
    #    
    #    if (obj-zopt)/zopt>0.1 && intsol==true && setdis==true
    #        println("Solucion satisface condiciones para agregar el escenario")
    #        return JuMP.StopTheSolver
    #    end
    #end
    
    #if time()-tt<=timemax
    #    addlazycallback(m,getcursol)
    #end
    #bbdata = NodeData[]
    
    
    #function infocallback(cb)
    #    obj       = MathProgBase.cbgetobj(cb)
    #    push!(bbdata, NodeData(time(),obj))
    #end
    #addinfocallback(m, infocallback, when = :MIPNode)
    
    #println(bbdata)

    status = solve(m)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    #println("Probando valor opt: ", valor)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end     
    return valor,cost_scen
end

SubproblemLPim (generic function with 1 method)

In [27]:
function solveRLPim(A,b,cminus, cplus, xmax, n::Int64,timemax,mindis,solver=CplexSolver(CPX_PARAM_LPMETHOD=1))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    @constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)        
    #print(m)
    
    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol#conteo<=10
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\t",fixup,"\n")
        #println("\t\t",fixun,"\n")
        if it==1
            tta=time()
            optimalcost,cost = SubproblemLP(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b)
            println("Tiempo subproblema: ", time()-tta)
            scens=cost
        else
            tta=time()
            optimalcost,cost = SubproblemLPim(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b,getobjectivevalue(m),time(),scens,timemax,mindis)
            println("Tiempo subproblema: ", time()-tta)
            scens=vcat(scens,cost)
        end
        println("Costo optimo subproblema: ", optimalcost)
        #New cut
        tt1=time()
        rhs=solveLP(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        println("Tiempo RHS: ", time()-tt1)
        @constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        println("Tiempo LP: ", time()-tt2)
        println(status)
        #println(getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        println("LB: ", getobjectivevalue(m))
        it += 1
        #print("GAP entre soluciones consecutivas", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

solveRLPim (generic function with 2 methods)

In [ ]:
solveRLPim(A,b,c_min, c_max, xmax, length(c_max),10,0.1)

CPXPARAM_LPMethod                                1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 1 columns.
Reduced LP has 400 rows, 1000 columns, and 398537 nonzeros.
Presolve time = 0.33 sec. (43.34 ticks)

Iteration log . . .
Iteration:     1    Scaled infeas =             0.000000
Iteration:     2    Objective     =          4622.312436
LB: 1.4432899320127035e-15
Prueba LB1.0000000000000013
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 2958 columns.
Aggregator did 14 substitutions.
Reduced MIP has 428 rows, 1028 columns, and 404165 nonzeros.
Reduced MIP has 14 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.63 sec. (82.76 ticks)
Probing time = 0.03 sec. (4.35 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: tradi


Mixed integer rounding cuts applied:  3

Root node processing (before b&c):
  Real time             =    9.21 sec. (1103.13 ticks)
Sequential b&c:
  Real time             =    0.18 sec. (14.13 ticks)
                          ------------
Total (root+branch&cut) =    9.39 sec. (1117.27 ticks)
Tiempo RHS: 1.6704260349273682
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             6.852794
Iteration:    44    Objective     =           124.883747
Tiempo LP: 0.32618005752563477
Optimal
LB: 23.48294337778791
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 75.7222.
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 981 columns.
Aggregator did 19

Tiempo RHS: 0.795464038848877

Implied bound cuts applied:  3
Mixed integer rounding cuts applied:  7

Root node processing (before b&c):
  Real time             =   14.28 sec. (1543.65 ticks)
Sequential b&c:
  Real time             =    1.54 sec. (162.94 ticks)
                          ------------
Total (root+branch&cut) =   15.82 sec. (1706.59 ticks)
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =            17.538814
Iteration:    30    Objective     =           130.843436
Tiempo LP: 0.2079639434814453
Optimal
LB: 26.138791576131204
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 81.9349.
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and

Tiempo LP: 0.26929196739196777
Optimal
LB: 27.498310585867817
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 84.2417.
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 978 columns.
Aggregator did 22 substitutions.
Reduced MIP has 444 rows, 3000 columns, and 407394 nonzeros.
Reduced MIP has 1978 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.64 sec. (83.24 ticks)
Probing time = 0.03 sec. (4.58 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.45 sec. (52.17 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  B

Tiempo subproblema: 19.557676935195923
Costo optimo subproblema: 83.19768012393189
Tiempo RHS: 0.272423828125

Implied bound cuts applied:  1
Mixed integer rounding cuts applied:  5
Gomory fractional cuts applied:  1

Root node processing (before b&c):
  Real time             =   17.52 sec. (2107.79 ticks)
Sequential b&c:
  Real time             =    1.02 sec. (108.66 ticks)
                          ------------
Total (root+branch&cut) =   18.54 sec. (2216.44 ticks)
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             3.439395
Iteration:    22    Objective     =            88.320514
Tiempo LP: 0.24818992614746094
Optimal
LB: 28.613375180034794
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP star

...

Tiempo subproblema: 44.37211399078369
Costo optimo subproblema: 31.86124396947833
Tiempo RHS: 0.4412112064361572

Implied bound cuts applied:  1
Mixed integer rounding cuts applied:  15
Gomory fractional cuts applied:  4

Root node processing (before b&c):
  Real time             =   35.92 sec. (3953.15 ticks)
Sequential b&c:
  Real time             =    7.56 sec. (754.46 ticks)
                          ------------
Total (root+branch&cut) =   43.47 sec. (4707.61 ticks)
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             0.013675
Iteration:    16    Objective     =            29.844123
Tiempo LP: 0.67806913948059082
Optimal
LB: 29.77600862715848
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP s

Tiempo LP: 0.7067340030670166
Optimal
LB: 29.786015179080945
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 30.5533.
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 969 columns.
Aggregator did 31 substitutions.
Reduced MIP has 462 rows, 3000 columns, and 411009 nonzeros.
Reduced MIP has 1969 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.62 sec. (91.74 ticks)
Probing time = 0.04 sec. (4.84 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 1.67 sec. (57.11 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Be

 
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 30.5848.
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 966 columns.
Aggregator did 34 substitutions.
Reduced MIP has 468 rows, 3000 columns, and 412223 nonzeros.
Reduced MIP has 1966 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.61 sec. (91.89 ticks)
Probing time = 0.03 sec. (4.90 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 1.76 sec. (58.40 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           30.5848       50.3135            64.51%
      0     0       33.5221     9       30.5848       33.5221       75    9.60%
      0     0       33.0749     7       30.5848    MIRcuts: 2       92    8.14%
      0   

Tiempo subproblema: 48.03001689910889
Costo optimo subproblema: 30.781018306765432
Tiempo RHS: 0.7649679183959961

Implied bound cuts applied:  3
Mixed integer rounding cuts applied:  12
Gomory fractional cuts applied:  3

Root node processing (before b&c):
  Real time             =   36.52 sec. (4008.28 ticks)
Sequential b&c:
  Real time             =   10.35 sec. (966.19 ticks)
                          ------------
Total (root+branch&cut) =   46.87 sec. (4974.47 ticks)
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             0.925769
Iteration:    77    Objective     =            49.045215
Removing shift (2).
Tiempo LP: 0.8546409797668457
Optimal
LB: 29.815056276536424
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
1 of 1 MIP starts prov

Tiempo LP: 0.34905809783935547
Optimal
LB: 29.82516361086668
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_Preprocessing_Reduce                    1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Strategy_CallbackReducedLP          0

************************ PIPA
 
